# Домашнее задание задача 1
Напишите функцию, которая возвращает название валюты (поле 'Name') с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [139]:
def max_valute():
    import requests
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    valute = dict(r.json()['Valute'])
    max_value=0
    max_valute=''
    for ind, val in valute.items():
        if (ind == 'XDR'): continue #Исключаем валюту "Специальные права заимствования"
        if (val['Value']/val['Nominal']>max_value):
            max_value=val['Value']/val['Nominal']
            max_valute=val['Name']
    return max_valute

max_valute()

'Фунт стерлингов Соединенного королевства'

# Домашнее задание задача 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [140]:
class Rate:
    def __init__(self, format_='value'):
        self.format = format_
        self.diff = False
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if (self.diff == True and (currency == 'EUR' or currency == 'USD')):
                    # на основании положения N 286-П Банка России от 18.04.2006 
                    return round(response[currency]['Value']-response[currency]['Previous'],4)
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self, diff_=False):
        """
        Возвращает курс евро на сегодня в формате self.format
        Если параметр self.diff=True и self.format='value', то возвращает 
        разницу курса евро с предыдущим значением self.format
        """
        if (diff_ == True):
            self.diff = diff_
        return self.make_format('EUR')
    
    def usd(self, diff_=False):
        """
        Возвращает курс доллара на сегодня в формате self.format
        Если параметр self.diff=True и self.format='value', то возвращает 
        разницу курса доллара с предыдущим значением self.format
        """
        if (diff_ == True):
            self.diff = diff_
        return self.make_format('USD')

In [141]:
Rate().usd(True)

0.6875

# Домашнее задание задача 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров. Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [142]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [143]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
        #устанавливаем начальный уровень сотрудника
        self.levelup=5
        self.grade+=seniority//self.levelup
    
    def check_if_it_is_time_for_upgrade(self):
        """
        Увеличение количества аккредитаций сотрудника-разработчика
        и при необходимости увеличение уровня сотрудника (при self.seniority кратному self.levelup)
        """
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % self.levelup == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [144]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority)
        #устанавливаем начальный уровень сотрудника
        self.levelup = 7
        self.awards = awards
        self.grade += (seniority + awards * 2) // self.levelup
   
    def check_if_it_is_time_for_upgrade(self):
        """
        Увеличение количества аккредитаций сотрудника-дизайнера
        и при необходимости увеличение уровня сотрудника (при self.seniority кратному self.levelup)
        """
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if (self.seniority + self.awards * 2) % self.levelup == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()
    
    def get_international_award(self):
        """
        Увеличение количества премий сотрудника-дизайнера
        и при необходимости увеличение уровня сотрудника (при self.seniority кратному self.levelup)
        """
        # для каждой премии увеличиваем счетчик на 1
        self.awards += 1
        
        # условие повышения сотрудника из презентации
        if ((self.seniority + self.awards * 2) % self.levelup == 0 
            or (self.seniority + self.awards * 2) % self.levelup == 1):
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [145]:
elena = Designer('Елена', seniority=0, awards=2)

In [146]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
